In [1]:
import json
import os
import shutil
import csv

In [2]:
fields = ['1Tainan', '3Taoyuan', '2Penghu']

In [3]:
# 南崁.的符號無法decode
bucket = ['220707_060614804_Camera_1', '220707_060617550_Camera_1', '220707_060722774_Camera_1', '220707_060801663_Camera_1', '220707_060950255_Camera_1', '220707_060951671_Camera_1', '220707_060952938_Camera_1', '220707_030343373_Camera_1', '220707_030715653_Camera_1', '220707_030716727_Camera_1', '220707_030717264_Camera_1', '220707_030728389_Camera_1', '220707_030728915_Camera_1', '220707_030728915_Camera_1', '220707_030729441_Camera_1', '220707_030729441_Camera_1', '220707_032755076_Camera_1', '220707_033559873_Camera_1', '220707_061120592_Camera_1', '220707_061123781_Camera_1', '220707_060908688_Camera_1']

# 圓形標記格式的檔名
bucket2 = ['221005_020123663_Camera_0', '221005_020124144_Camera_0', '221005_020124604_Camera_0', '221005_020125040_Camera_0', '221005_020129529_Camera_0', '221005_020130027_Camera_0', '221005_020130531_Camera_0', '221005_020131006_Camera_0', '221005_020131457_Camera_0', '221005_020131893_Camera_0', '221005_020132299_Camera_0', '221005_020132685_Camera_0', '221005_020133047_Camera_0', '221005_020133388_Camera_0', '221005_020133712_Camera_0', '221005_020134020_Camera_0', '220707_030128789_Camera_1', '220707_055132895_Camera_1']

# rect標記格式的檔名
bucket3 = ['220707_030651749_Camera_1', '220707_030652867_Camera_1', ]


In [4]:
for field in fields:
    img_folder_path = f"C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\{field}\\images"
    annotation_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\{field}\\{field}_ori_all.json'
    target_folder_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}'
    csv_file_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\x_labels.csv'

    os.makedirs(target_folder_path, exist_ok = True)
    with open(csv_file_path, 'w', newline='', encoding='big5') as csvfile:
        print('hi')
        fieldnames = ['raw_filename', 'filename', 'category', 'box', 'words']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with open(annotation_path, 'r', encoding='utf-8') as f:
            annotations = json.load(f)

            can = []
            rawdata = []
            # 每張圖
            for image_key, annotation in annotations.items():
                image_filename = annotation["filename"]
                if image_filename.split('.')[0] not in bucket2 and image_filename.split('.')[0] not in bucket3:
                    image_path = os.path.join(img_folder_path, image_filename)

                    regions = annotation["regions"]
                    valid_regions = []

                    #每個標記
                    for region in regions:         
                        all_points_x = region['shape_attributes'].get('all_points_x', [])
                        all_points_y = region['shape_attributes'].get('all_points_y', [])
                        try:
                            box = list(zip(all_points_x, all_points_y))
                            width = max(all_points_x) - min(all_points_x)
                            height = max(all_points_y) - min(all_points_y)
                            ratio = width / height
                        except:
                            print(f'{image_filename=}, {region=}')

                        category = region['region_attributes']['category']  

                        # 小符號組
                        if str(category).split('_')[0] in ['B']:
                            word = region['region_attributes'].get('word', '')
                            if '*' in word:
                                can.append(image_filename)
                            if word == '':
                                can.append(image_filename)
                            if '*' not in word and word != '':
    #                             print('小符號組append一個')
                                writer.writerow({
                                    'raw_filename': image_filename,
                                    'filename': '',
                                    'category': category,
                                    'box': box,
                                    'words': word
                                })

                        # 限制
                        elif str(category).split('_')[0] in ['7', '8', '9', '10', '11', '12']:
                            try:
                                word = str(category).split('_')[1]
                                if '*' in word:
                                    can.append(image_filename)
                                if word == '':
                                    can.append(image_filename)
                                if '*' not in word and word != '':
    #                                 print('限制append一個')
                                    writer.writerow({
                                        'raw_filename': image_filename,
                                        'filename': '',
                                        'category': category,
                                        'box': box,
                                        'words': word
                                    })
                            except IndexError as e:
                                # print(f"{image_filename}之中的標記 {category} 錯誤: {e}")
                                can.append(image_filename)

                        # 主要文字路牌
                        elif str(category) in ['1', '2', '3', '4', '5', '6'] and ratio > 1.5:
                            word = region['region_attributes'].get('word', '')
                            if '*' not in word and any('\u4e00' <= char <= '\u9fff' for char in word):
                                rawdata.append(image_filename)
    #                             print('主要文字路牌append一個')
                                writer.writerow({
                                        'raw_filename': image_filename,
                                        'filename': '',
                                        'category': category,
                                        'box': box,
                                        'words': word
                                    })
                                # print(f"{image_filename}的標記 {category} word沒有中文字:{word}")
                            distance = region['region_attributes'].get('distance', '')

                    # if distance != '':
                         # print(image_filename, distance)


                if image_filename in rawdata and image_filename not in can:
                    target_path = os.path.join(target_folder_path, image_filename)
                    shutil.copy(image_path, target_path)


hi
hi
hi
